In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

## Dataset

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|██████████| 26421880/26421880 [00:10<00:00, 2585441.13it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 1623475.57it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:01<00:00, 2834242.51it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 43620761.60it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)


In [5]:
train_dataloader.dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

## nn Model

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print("Device is {}".format(device))

Device is mps


In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# create nn model    
model = NeuralNetwork().to(device)
print(model) 

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Training model

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # BP and update w
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [14]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1} \n ----------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("\n Training Done !")


Epoch 1 
 ----------------------
loss: 1.911541  [   64/60000]
loss: 1.875595  [ 6464/60000]
loss: 1.748466  [12864/60000]
loss: 1.804434  [19264/60000]
loss: 1.676145  [25664/60000]
loss: 1.630790  [32064/60000]
loss: 1.676788  [38464/60000]
loss: 1.570385  [44864/60000]
loss: 1.599013  [51264/60000]
loss: 1.493911  [57664/60000]
Test Error: 
 Accuracy: 61.1%, Avg loss: 1.499019 

Epoch 2 
 ----------------------
loss: 1.572172  [   64/60000]
loss: 1.533181  [ 6464/60000]
loss: 1.371266  [12864/60000]
loss: 1.452154  [19264/60000]
loss: 1.317897  [25664/60000]
loss: 1.316194  [32064/60000]
loss: 1.350291  [38464/60000]
loss: 1.272870  [44864/60000]
loss: 1.299652  [51264/60000]
loss: 1.204653  [57664/60000]
Test Error: 
 Accuracy: 63.4%, Avg loss: 1.223425 

Epoch 3 
 ----------------------
loss: 1.303215  [   64/60000]
loss: 1.290115  [ 6464/60000]
loss: 1.109710  [12864/60000]
loss: 1.222985  [19264/60000]
loss: 1.087919  [25664/60000]
loss: 1.110712  [32064/60000]
loss: 1.154499  [

## Saving Model

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved Model State to model.pth")

Saved Model State to model.pth


## Loading Model

In [16]:
load_model = NeuralNetwork().to(device)
load_model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [17]:
load_model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [19]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    print(f'Predicted: "{pred[0].argmax(0)}", Actual: "{y}"')



Predicted: "9", Actual: "9"
